In [1]:
import polars as pl
pl.__version__

'0.19.13'

In [2]:
df = pl.scan_csv("TA.csv", try_parse_dates=True)

In [3]:
df.limit(0).collect().schema

OrderedDict([('NAME', Utf8),
             ('Date', Date),
             ('Median_SP', Float64),
             ('Median_SP_Real', Float64),
             ('Fisher_PI', Float64),
             ('Quantity', Int64),
             ('Stock', Int64),
             ('Quantity_Stock', Float64),
             ('spar_chained', Utf8),
             ('spar_chained_land', Utf8),
             ('LV_CV_mean', Utf8),
             ('LV_mean', Utf8),
             ('Geomean_Rent', Int64),
             ('Geomean_Rent_Real', Float64),
             ('Rent_Fisher_PI', Float64),
             ('Rent_ratio', Float64),
             ('HAM_Buy', Utf8),
             ('HAM_Rent', Utf8)])

In [29]:

q = (
    df  .select(['NAME', 'Date', 'Median_SP'])
        .filter(pl.col('NAME') == 'Greater Christchurch')
        .drop_nulls()
        .group_by(pl.col('Date').dt.year(), maintain_order=True)
        .agg(
            pl.col('NAME').first().alias("Region"), 
            pl.col('Median_SP').mean().round().alias('Median_Price'), 
            pl.col('Median_SP').count().alias('Observations')
        )
    )

res = q.collect()
res


Date,Region,Median_Price,Observations
i32,str,f64,u32
1993,"""Greater Christ…",116750.0,4
1994,"""Greater Christ…",126250.0,4
1995,"""Greater Christ…",131375.0,4
1996,"""Greater Christ…",139250.0,4
1997,"""Greater Christ…",147352.0,4
1998,"""Greater Christ…",149788.0,4
1999,"""Greater Christ…",154625.0,4
2000,"""Greater Christ…",155000.0,4
2001,"""Greater Christ…",156688.0,4
